In [2]:
import requests
import pandas as pd
import time

class SingaporeData:
    def __init__(self):
        self.base_url = 'https://api-production.data.gov.sg/v2/public/api/'
        self.collection_id = 189
        self.data = self._get_all_records()
        
    def _get_dateset_ids(self):
        """Fetch the dataset IDs from the API."""
        collection_url = f"collections/{self.collection_id}/metadata"
        response = requests.get(self.base_url + collection_url)
        response.raise_for_status()
        return response.json()['data']['collectionMetadata']['childDatasets']
    
    def _get_all_records(self, max_polls=5, delay=5):
        """Fetch and concatenate all datasets into a single DataFrame."""
        list_of_dfs = []
        dataset_ids = self._get_dateset_ids()
        
        for dataset_id in dataset_ids:
            for attempt in range(1, max_polls + 1):
                response = requests.get(
                    f"https://api-open.data.gov.sg/v1/public/api/datasets/{dataset_id}/poll-download",
                    headers={'Content-Type': 'application/json'},
                    json={}
                )
                data = response.json().get('data', {})
                
                if 'url' in data:
                    try:
                        df = pd.read_csv(data['url'])
                        list_of_dfs.append(df)
                        break
                    except Exception as e:
                        print(f"Error loading data for dataset {dataset_id}: {e}")
                        break
                else:
                    if attempt == max_polls:
                        print(f"Failed to retrieve download URL for dataset {dataset_id} after {max_polls} attempts.")
                    else:
                        print(f"No result yet for dataset {dataset_id}. Retrying in {delay} seconds.")
                        time.sleep(delay)

        if list_of_dfs:
            return pd.concat(list_of_dfs, ignore_index=True)
        else:
            print("No data available to concatenate.")
            return pd.DataFrame()

# Instantiate the class and access the concatenated DataFrame
singapore = SingaporeData()
final_data = singapore.data  # Access the concatenated DataFrame


In [3]:
final_data.head()

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price
0,2017-01,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,10 TO 12,44.0,Improved,1979,61 years 04 months,232000.0
1,2017-01,ANG MO KIO,3 ROOM,108,ANG MO KIO AVE 4,01 TO 03,67.0,New Generation,1978,60 years 07 months,250000.0
2,2017-01,ANG MO KIO,3 ROOM,602,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,262000.0
3,2017-01,ANG MO KIO,3 ROOM,465,ANG MO KIO AVE 10,04 TO 06,68.0,New Generation,1980,62 years 01 month,265000.0
4,2017-01,ANG MO KIO,3 ROOM,601,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,265000.0


In [4]:
final_data.isnull().sum()

month                       0
town                        0
flat_type                   0
block                       0
street_name                 0
storey_range                0
floor_area_sqm              0
flat_model                  0
lease_commence_date         0
remaining_lease        709050
resale_price                0
dtype: int64

In [45]:
df = final_data.copy()

In [41]:
df.isnull().sum()

month                       0
town                        0
flat_type                   0
block                       0
street_name                 0
storey_range                0
floor_area_sqm              0
flat_model                  0
lease_commence_date         0
remaining_lease        709050
resale_price                0
dtype: int64

In [8]:
df['lease_end_date'] = df['lease_commence_date'] + 99


In [18]:
df.head()

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price
0,2017-01,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,10 TO 12,44.0,Improved,1979,61 years 04 months,232000.0
1,2017-01,ANG MO KIO,3 ROOM,108,ANG MO KIO AVE 4,01 TO 03,67.0,New Generation,1978,60 years 07 months,250000.0
2,2017-01,ANG MO KIO,3 ROOM,602,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,262000.0
3,2017-01,ANG MO KIO,3 ROOM,465,ANG MO KIO AVE 10,04 TO 06,68.0,New Generation,1980,62 years 01 month,265000.0
4,2017-01,ANG MO KIO,3 ROOM,601,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,265000.0


In [15]:
df['year'] = pd.to_datetime(df['month']).dt.year

In [39]:
for feature in df.columns:
    print(feature ,"unique are ",df[feature].nunique())

month unique are  419
town unique are  27
flat_type unique are  8
block unique are  2729
street_name unique are  588
storey_range unique are  25
floor_area_sqm unique are  218
flat_model unique are  21
lease_commence_date unique are  56
remaining_lease unique are  61
resale_price unique are  9734
year unique are  35
lease_end_date unique are  61


In [46]:
def correct_data(df):

    df['year'] = pd.to_datetime(df['month']).dt.year
    df.drop(columns='month',inplace=True)
    df['remaining_lease'] = df['lease_commence_date']+99-df['year']
    df['flat_model'] = df['flat_model'].str.title()


    return df


In [47]:
correct_data(df)

KeyError: "['month'] not found in axis"

In [30]:
df.isnull().sum()

month                  0
town                   0
flat_type              0
block                  0
street_name            0
storey_range           0
floor_area_sqm         0
flat_model             0
lease_commence_date    0
remaining_lease        0
resale_price           0
year                   0
lease_end_date         0
dtype: int64

In [34]:
df['flat_model'].str.title().unique()

array(['Improved', 'New Generation', 'Dbss', 'Standard', 'Apartment',
       'Simplified', 'Model A', 'Premium Apartment', 'Adjoined Flat',
       'Model A-Maisonette', 'Maisonette', 'Type S1', 'Type S2',
       'Model A2', 'Terrace', 'Improved-Maisonette', 'Premium Maisonette',
       'Multi Generation', 'Premium Apartment Loft', '2-Room', '3Gen'],
      dtype=object)

In [35]:
df['flat_model'] = df['flat_model'].str.title()